#  Import images

In [1]:
#
# Term project for AMLS_ELEC0134
#
%matplotlib inline
import matplotlib.pyplot as mplp
import matplotlib.image as mpimg
from keras.preprocessing import image
from PIL import Image
import cv2
import numpy
import os
import pandas
from sklearn.metrics import confusion_matrix
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

'''imagePath = './dataset/image/IMAGE_0574.jpg'
imgFile = cv2.imread(imagePath,0)
#print(imgFile.shape)
minVal=numpy.amin(imgFile)
#print('min= ',minVal)
maxVal= numpy.amax(imgFile)
#print('max= ',maxVal)
#mplp.imshow(imgFile)
#mplp.show()'''

#
# load the images
target_size=[64,64]

training_images_dir='./trainingDataset/image'
training_images_paths = [os.path.join(training_images_dir,l) for l in os.listdir(training_images_dir)]
training_images = []
#print(training_images_paths)
for img_path in training_images_paths:
    filename=img_path.split('\\')[-1]
    #print('filename=',filename)
    #target_size=None
    img = image.img_to_array(image.load_img(img_path,target_size=target_size, interpolation='bicubic'))
    #img = image.img_to_array(image.load_img(img_path,target_size=[64,64], interpolation='bicubic'))
    imgBW = img[:,:,0]
    #print(img.shape)
    #print(imgBW.shape)
    '''if (filename=='IMAGE_0000.jpg'):
        mplp.imshow(img/255)
        mplp.show()
        mplp.imshow(imgBW/255)
        mplp.show()'''
    training_images.append(imgBW)
trainingImages=numpy.array(training_images)
print(trainingImages.shape)

test_images_dir='./testDataset/image'
test_images_paths = [os.path.join(test_images_dir,l) for l in os.listdir(test_images_dir)]
test_images = []
for img_path in test_images_paths:
    filename=img_path.split('\\')[-1]
    #print('filename=',filename)
    #target_size=None
    img = image.img_to_array(image.load_img(img_path,target_size=target_size, interpolation='bicubic'))
    #img = image.img_to_array(image.load_img(img_path,target_size=[64,64], interpolation='bicubic'))
    imgBW = img[:,:,0]
    #print(img.shape)
    #print(imgBW.shape)
    '''if (filename=='IMAGE_0000.jpg'):
        mplp.imshow(img/255)
        mplp.show()
        mplp.imshow(imgBW/255)
        mplp.show()'''
    test_images.append(imgBW)
testImages=numpy.array(test_images)
print(testImages.shape)

Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term
(3000, 64, 64)
(200, 64, 64)


# Import tumor labels

In [2]:
trainingTumorLabels= pandas.read_csv('./trainingDataset/label.csv')
#print(trainingTumorLabels.shape)
#print(trainingTumorLabels)
trainingTumorLabels=trainingTumorLabels['label']
#print(trainingTumorLabels.shape)
#print(trainingTumorLabels)
'''tumorNumbers = [float(0.0) if label=='no_tumor' else float(1.0) for label in tumorLabels]
for (i,label) in enumerate(tumorLabels):
    if label=='glioma_tumor':
        tumorNumbers[i]=1.0
    else: 
        if label == 'meningioma_tumor':
            tumorNumbers[i]=2.0
        else:
            if label == 'pituitary_tumor':
                tumorNumbers[i]=3.0
            else:
                tumorNumbers[i]=0.0'''
trainingTumorNumbers = [0 if label=='no_tumor' else 1 for label in trainingTumorLabels]
trainingTumorNumbers = numpy.array(trainingTumorNumbers)
print('trainingTumorNumbers is ',trainingTumorNumbers.dtype)
print('trainingTumorNumbers is ',trainingTumorNumbers.shape)
#print(trainingTumorNumbers)
trainingTumorClasses = numpy.array([trainingTumorNumbers, 1-trainingTumorNumbers]).T
print(trainingTumorClasses.shape)
print(trainingTumorClasses)


testTumorLabels= pandas.read_csv('./testDataset/label.csv')
#print(testTumorLabels.shape)
#print(testTumorLabels)
testTumorLabels=testTumorLabels['label']
#print(testTumorLabels.shape)
#print(testTumorLabels)
'''tumorNumbers = [float(0.0) if label=='no_tumor' else float(1.0) for label in tumorLabels]
for (i,label) in enumerate(tumorLabels):
    if label=='glioma_tumor':
        tumorNumbers[i]=1.0
    else: 
        if label == 'meningioma_tumor':
            tumorNumbers[i]=2.0
        else:
            if label == 'pituitary_tumor':
                tumorNumbers[i]=3.0
            else:
                tumorNumbers[i]=0.0'''
testTumorNumbers = [0 if label=='no_tumor' else 1 for label in testTumorLabels]
testTumorNumbers = numpy.array(testTumorNumbers)
print('testTumorNumbers is ',testTumorNumbers.dtype)
print('testTumorNumbers is ',testTumorNumbers.shape)
#print(testTrainingTumorNumbers)
testTumorClasses = numpy.array([testTumorNumbers, 1-testTumorNumbers]).T
print(testTumorClasses.shape)
print(testTumorClasses)


trainingTumorNumbers is  int32
trainingTumorNumbers is  (3000,)
(3000, 2)
[[1 0]
 [0 1]
 [1 0]
 ...
 [1 0]
 [1 0]
 [1 0]]
testTumorNumbers is  int32
testTumorNumbers is  (200,)
(200, 2)
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [

# Define MLP

In [3]:
def allocate_weights_and_biases_old():

    # define number of hidden layers ..
    n_hidden_1 = 2048  # 1st layer number of neurons
    n_hidden_2 = 2048  # 2nd layer number of neurons

    # inputs placeholders
    #X = tf.placeholder("float", [None, 68, 2])
    X = tf.placeholder("float", [None, 64, 64])
    Y = tf.placeholder("float", [None, 2])  # 2 output classes
    #Y = tf.placeholder("float", [None, 4])  # 4 output classes
    
    # flatten image features into one vector (i.e. reshape image feature matrix into a vector)
    images_flat = tf.layers.flatten(X)  # This generates a warning
    #images_flat = tf.keras.layers.Flatten(X) # this doesn't work
    
    # weights and biases are initialized from a normal distribution with a specified standard devation stddev
    stddev = 0.01
    
    # define placeholders for weights and biases in the graph
    weights = {
        'hidden_layer1': tf.Variable(tf.random_normal([64*64, n_hidden_1], stddev=stddev)), # images are 64x64
        'hidden_layer2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
        'out': tf.Variable(tf.random_normal([n_hidden_2, 2], stddev=stddev)) 
    }

    biases = {
        'bias_layer1': tf.Variable(tf.random_normal([n_hidden_1], stddev=stddev)),
        'bias_layer2': tf.Variable(tf.random_normal([n_hidden_2], stddev=stddev)),
        'out': tf.Variable(tf.random_normal([2], stddev=stddev))
    }
    
    return weights, biases, X, Y, images_flat

In [4]:
# Create model
def multilayer_perceptron_old():
        
    weights, biases, X, Y, images_flat = allocate_weights_and_biases()

    # Hidden fully connected layer 1
    layer_1 = tf.add(tf.matmul(images_flat, weights['hidden_layer1']), biases['bias_layer1'])
    layer_1 = tf.sigmoid(layer_1)

    # Hidden fully connected layer 2
    layer_2 = tf.add(tf.matmul(layer_1, weights['hidden_layer2']), biases['bias_layer2'])
    layer_2 = tf.sigmoid(layer_2)
    
    # Output fully connected layer
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']

    return out_layer, X, Y

# Define an MLP with 0-4 hidden layers, input number of neurons in each layer

In [5]:

def allocate_weights_and_biases(neurons_hidden):
    
    # define number of hidden layers ..
    num_layers = len(neurons_hidden)     

    # inputs placeholders
    X = tf.placeholder("float", [None, 64,64]) # images are 64x64
    Y = tf.placeholder("float", [None, 2])  # 2 output classes
    
    # flatten image features into one vector (i.e. reshape image feature matrix into a vector)
    images_flat = tf.layers.flatten(X)  
    #images_flat = tf.keras.layers.Flatten(X) # this doesn't work
    
    # weights and biases are initialized from a normal distribution with a specified standard devation stddev
    stddev = 0.01
    
    # define placeholders for weights and biases in the graph
    if num_layers==0:
        weights = {
            'out': tf.Variable(tf.random_normal([64*64, 2], stddev=stddev))
        }
        biases = {
            'out': tf.Variable(tf.random_normal([2], stddev=stddev))
        }
    if num_layers==1:
        weights = {
            'hidden_layer1': tf.Variable(tf.random_normal([64*64, neurons_hidden[0]], stddev=stddev)),
            'out': tf.Variable(tf.random_normal([neurons_hidden[num_layers-1], 2], stddev=stddev))
        }
        biases = {
            'bias_layer1': tf.Variable(tf.random_normal([neurons_hidden[0]], stddev=stddev)),
            'out': tf.Variable(tf.random_normal([2], stddev=stddev))
        }
        
    if num_layers==2:
        weights = {
            'hidden_layer1': tf.Variable(tf.random_normal([64*64, neurons_hidden[0]], stddev=stddev)),
            'hidden_layer2': tf.Variable(tf.random_normal([neurons_hidden[0], neurons_hidden[1]], stddev=stddev)),
            'out': tf.Variable(tf.random_normal([neurons_hidden[num_layers-1], 2], stddev=stddev))
        }
        biases = {
            'bias_layer1': tf.Variable(tf.random_normal([neurons_hidden[0]], stddev=stddev)),
            'bias_layer2': tf.Variable(tf.random_normal([neurons_hidden[1]], stddev=stddev)),
            'out': tf.Variable(tf.random_normal([2], stddev=stddev))
        }
              
    if num_layers==3:
        weights = {
            'hidden_layer1': tf.Variable(tf.random_normal([64*64, neurons_hidden[0]], stddev=stddev)),
            'hidden_layer2': tf.Variable(tf.random_normal([neurons_hidden[0], neurons_hidden[1]], stddev=stddev)),
            'hidden_layer3': tf.Variable(tf.random_normal([neurons_hidden[1], neurons_hidden[2]], stddev=stddev)),
            'out': tf.Variable(tf.random_normal([neurons_hidden[num_layers-1], 2], stddev=stddev))
        }
        biases = {
            'bias_layer1': tf.Variable(tf.random_normal([neurons_hidden[0]], stddev=stddev)),
            'bias_layer2': tf.Variable(tf.random_normal([neurons_hidden[1]], stddev=stddev)),
            'bias_layer3': tf.Variable(tf.random_normal([neurons_hidden[2]], stddev=stddev)),
            'out': tf.Variable(tf.random_normal([2], stddev=stddev))
        }
           
    if num_layers==4:
        weights = {
            'hidden_layer1': tf.Variable(tf.random_normal([64*64, neurons_hidden[0]], stddev=stddev)),
            'hidden_layer2': tf.Variable(tf.random_normal([neurons_hidden[0], neurons_hidden[1]], stddev=stddev)),
            'hidden_layer3': tf.Variable(tf.random_normal([neurons_hidden[1], neurons_hidden[2]], stddev=stddev)),
            'hidden_layer4': tf.Variable(tf.random_normal([neurons_hidden[2], neurons_hidden[3]], stddev=stddev)),
            'out': tf.Variable(tf.random_normal([neurons_hidden[num_layers-1], 2], stddev=stddev))
        }
        biases = {
            'bias_layer1': tf.Variable(tf.random_normal([neurons_hidden[0]], stddev=stddev)),
            'bias_layer2': tf.Variable(tf.random_normal([neurons_hidden[1]], stddev=stddev)),
            'bias_layer3': tf.Variable(tf.random_normal([neurons_hidden[2]], stddev=stddev)),
            'bias_layer4': tf.Variable(tf.random_normal([neurons_hidden[3]], stddev=stddev)),
            'out': tf.Variable(tf.random_normal([2], stddev=stddev))
        }
    
    return weights, biases, X, Y, images_flat
    

In [6]:
# Create model
def multilayer_perceptron(neurons_hidden):
        
    num_layers = len(neurons_hidden) 
    weights, biases, X, Y, images_flat = allocate_weights_and_biases(neurons_hidden=neurons_hidden)

    # Hidden fully connected layer 1
    layer_1 = tf.add(tf.matmul(images_flat, weights['hidden_layer1']), biases['bias_layer1'])
    layer_1 = tf.sigmoid(layer_1)
    layer_final = layer_1

    # Hidden fully connected layer 2
    if (num_layers>=2):
        layer_2 = tf.add(tf.matmul(layer_1, weights['hidden_layer2']), biases['bias_layer2'])
        layer_2 = tf.sigmoid(layer_2)
        layer_final = layer_2

    # Hidden fully connected layer 3
    if (num_layers>=3):
        layer_3 = tf.add(tf.matmul(layer_2, weights['hidden_layer3']), biases['bias_layer3'])
        layer_3 = tf.sigmoid(layer_3)
        layer_final = layer_3

    # Hidden fully connected layer 4
    if (num_layers>=4):
        layer_4 = tf.add(tf.matmul(layer_3, weights['hidden_layer4']), biases['bias_layer4'])
        layer_4 = tf.sigmoid(layer_4)
        layer_final = layer_4                     
    
    # Output fully connected layer
    out_layer = tf.matmul(layer_final, weights['out']) + biases['out']

    return out_layer, X, Y



# Initialization

In [8]:
# learning parameters
learning_rate = 1e-5
training_epochs = 500
#neurons_hidden = [512, 128, 32, 8]
#neurons_hidden = [2048,512, 64, 8]
#neurons_hidden = [2048,2048,256,32]
neurons_hidden = [2048,2048,2048,2048]
#neurons_hidden = [16,4]
#neurons_hidden = [4096]
#neurons_hidden = [16*1024]
num_layers = len(neurons_hidden) # default
num_layers = 4  # override
neurons_hidden = neurons_hidden[:num_layers]
print(num_layers,' layers')
print(neurons_hidden,' neurons')

# display training accuracy every ..
display_accuracy_step = 10

# set size of training and test sets
#trainingSize = 2000
#testSize=1000

#training_images, training_labels, test_images, test_labels = get_data()
#training_images = allImages[:trainingSize]
#print(training_images.shape)
#test_images = allImages[trainingSize:trainingSize+testSize]
#print(test_images.shape)
#trainingClasses = tumorClasses[:trainingSize]
#print(trainingClasses.T)
#testClasses = tumorClasses[trainingSize:trainingSize+testSize]
#print(testClasses.T)
logits, X, Y = multilayer_perceptron(neurons_hidden=neurons_hidden)

# define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

# define training graph operation
train_op = optimizer.minimize(loss_op)

# graph operation to initialize all variables
init_op = tf.global_variables_initializer()


4  layers
[2048, 2048, 2048, 2048]  neurons
Instructions for updating:
Use keras.layers.Flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



# Run the sessions

In [11]:
with tf.Session() as sess:
    num_sessions = 2
    accuracy_eval = numpy.zeros(num_sessions)
    
    for session in range(num_sessions):
        # run graph weights/biases initialization op
        sess.run(init_op)
        # begin training loop ..
        for epoch in range(training_epochs):
            # complete code below
            # run optimization operation (backprop) and cost operation (to get loss value)
            #_, cost = sess.run([train_op, loss_op], feed_dict={X: training_images,
                                                               #Y: trainingClasses})
            _, cost = sess.run([train_op, loss_op], feed_dict={X: trainingImages,
                                                               Y: trainingTumorClasses})

            # Display logs per epoch step
            #print("Epoch:", '%04d' % (epoch + 1), "cost={:.9f}".format(cost))
                
            if epoch % display_accuracy_step == 0:
                pred = tf.nn.softmax(logits)  # Apply softmax to logits
                correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))

                # calculate training accuracy
                accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
                #print("Accuracy: {:.3f}".format(accuracy.eval({X: training_images, Y: trainingClasses})))

        print(session+1," Optimization Finished!")

        # -- Define and run test operation -- #
        
        # apply softmax to output logits
        pred = tf.nn.softmax(logits)
        
        #  derive inffered calasses as the class with the top value in the output density function
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
        
        # calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            
        # run test accuracy operation ..
        accuracy_eval[session] = accuracy.eval({X: testImages, Y: testTumorClasses})
        print(session+1," Test Accuracy:", accuracy_eval[session])
        #print("Test Accuracy:", accuracy.eval({X: test_images, Y: testClasses}))
        
        predictions = pred.eval(feed_dict = {X:testImages})
        print(predictions.T)
        predictions_onehot = predictions.copy()
        for i, prediction in enumerate(predictions):
            #print(i,'   ',prediction)
            if prediction[0] > prediction[1]:
                predictions_onehot[i] = [1,0]
            else:
                predictions_onehot[i] = [0,1]
        print(predictions_onehot.T)
        
    print('Mean Test Accuracy = ',numpy.mean(accuracy_eval))
      



1  Optimization Finished!
1  Test Accuracy: 0.925000011920929
[[9.9902427e-01 9.9884307e-01 9.9130589e-01 9.9855584e-01 9.9856335e-01
  9.9897087e-01 9.9893039e-01 9.9876702e-01 9.7529560e-01 5.9051746e-01
  4.9829662e-01 8.2141882e-01 9.9893433e-01 6.9798172e-01 9.9692160e-01
  9.9877018e-01 9.8575181e-01 9.9859256e-01 4.4516394e-01 9.9352503e-01
  9.9904448e-01 9.9883634e-01 9.9864918e-01 9.9893004e-01 9.9868566e-01
  9.9873096e-01 1.3977438e-01 9.9854517e-01 8.8259988e-02 9.9816114e-01
  5.1205568e-02 7.2560835e-01 9.9861050e-01 9.9865025e-01 9.6658695e-01
  9.0957022e-01 9.9834657e-01 9.9485153e-01 9.9878234e-01 9.9856323e-01
  5.3216420e-02 1.7828327e-02 8.0593240e-01 9.7015184e-01 9.9818307e-01
  9.9881905e-01 9.9800986e-01 9.9410808e-01 9.8682982e-01 5.0282203e-02
  3.2082316e-02 9.9868363e-01 9.9900275e-01 9.9788767e-01 9.9818569e-01
  9.9901116e-01 9.9834204e-01 9.9875212e-01 9.9856025e-01 9.9707174e-01
  9.9896526e-01 9.9880731e-01 9.9844474e-01 9.9773586e-01 9.9889475e-01
  

In [13]:
   #print(testClasses.T)
   #print(predictions_onehot.T)
   print('Confusion Matrix')
   print('Actual values in X')
   print('Predicted values in Y')
   confusionMatrix = confusion_matrix(predictions_onehot.T[0],testTumorClasses.T[0])
   print(confusionMatrix,'\n')
 
   inputNumbers = numpy.sum(confusionMatrix, axis=0)
   print(inputNumbers[0],' cases of no tumor: ',round(100*confusionMatrix[0,0]/inputNumbers[0],1),'% chance of correct prediction')
   print(inputNumbers[1],' cases of a tumor: ',round(100*confusionMatrix[1,1]/inputNumbers[1],1),'% chance of correct prediction\n') 

   outputNumbers = numpy.sum(confusionMatrix, axis=1)
   print(outputNumbers[0],' predictions of no tumor are correct ',round(100*confusionMatrix[0,0]/outputNumbers[0],1),'% of the time')
   print(outputNumbers[1],' predictions of a tumor are correct ',round(100*confusionMatrix[1,1]/outputNumbers[1],1),'% of the time')
   




Confusion Matrix
Actual values in X
Predicted values in Y
[[ 23   4]
 [ 14 159]] 

37  cases of no tumor:  62.2 % chance of correct prediction
163  cases of a tumor:  97.5 % chance of correct prediction

27  predictions of no tumor are correct  85.2 % of the time
173  predictions of a tumor are correct  91.9 % of the time
